# Installing Libraries

In [ ]:
!pip install https://github.com/pypa/pip/archive/master.zip
!pip install --quiet gradio
!pip install git+https://github.com/LIAAD/yake
!pip install flashtext
!pip install transformers
!pip install nltk
!python -m spacy download en
!pip install spacy==2.1.3 --upgrade --force-reinstall
!pip install pywsd==1.0.2
!pip install -U wn==0.0.23
!pip install bert-extractive-summarizer --upgrade --force-reinstall
!pip install sentence-transformers==0.2.5.1
!pip install benepar==0.1.2
!pip install summa
!pip install scipy
!pip install transformers==2.6.0
!pip install torch==1.4.0
!pip install tensorflow==1.14.0
!pip install markupsafe==2.0.1
!pip install tabulate

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Importing Libraries

In [ ]:
import string
import re
import string
import itertools
import statistics
import csv
import os
import torch
import time
import random
import yake
import transformers
import benepar
import scipy
import gradio as gr
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('popular')
from nltk import tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from statistics import mode
from tabulate import tabulate
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from flashtext import KeywordProcessor
from collections import namedtuple
from tabulate import tabulate
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import BertTokenizer
from transformers import pipeline
from summa.summarizer import summarize
from string import punctuation
benepar.download('benepar_en2')
benepar_parser = benepar.Parser("benepar_en2")
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer

# Pre-Processing

In [ ]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

In [ ]:
def get_keywords(text):
  lem = WordNetLemmatizer()
  stoplist = list(string.punctuation)
  stoplist += stopwords.words('english')
  kw_extractor = yake.KeywordExtractor(lan='en', n=2, dedupFunc='0.9', top=50, stopwords=stoplist)
  keywords = kw_extractor.extract_keywords(text)
  res = [item for t in keywords for item in t]
  keywords = [i for j, i in enumerate(res) if j % 2 == 0]
  keywords_new = []
  lemm = []
  for k in keywords:
    w = word_tokenize(k)
    pos = pos_tag(w)
    flag = 0
    for i in range(len(pos)):
      if pos[i][1]=='NN' or pos[i][1]=='NNS' or pos[i][1]=='NNP' or pos[i][1]=='NNPS':
        continue
      else:
        flag = 1
        break
    if flag == 0:
      if lem.lemmatize(k) not in lemm:
        keywords_new.append(k)
        lemm.append(lem.lemmatize(k))
  return keywords_new

In [ ]:
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)
    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=False)
        keyword_sentences[key] = values
    return keyword_sentences

# True or False

In [ ]:
def preprocess(sentences):
  output = []
  for sent in sentences:
      single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
      double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
      question_present = "?" in sent
      if single_quotes_present or double_quotes_present or question_present :
          continue
      else:
          output.append(sent.strip(punctuation))
  return output
def get_candidate_sents(resolved_text, ratio=0.5): #change
  candidate_sents = summarize(resolved_text, ratio=ratio)
  candidate_sents_list = tokenize.sent_tokenize(candidate_sents)
  candidate_sents_list = [re.split(r'[:;]+',x)[0] for x in candidate_sents_list ]
  filtered_list_short_sentences = [sent for sent in candidate_sents_list if len(sent)<150] #change
  return filtered_list_short_sentences

In [ ]:
def get_flattened(t):
  sent_str_final = None
  if t is not None:
      sent_str = [" ".join(x.leaves()) for x in list(t)]
      sent_str_final = [" ".join(sent_str)]
      sent_str_final = sent_str_final[0]
  return sent_str_final

def get_termination_portion(main_string,sub_string):
  combined_sub_string = sub_string.replace(" ","")
  main_string_list = main_string.split()
  last_index = len(main_string_list)
  for i in range(last_index):
      check_string_list = main_string_list[i:]
      check_string = "".join(check_string_list)
      check_string = check_string.replace(" ","")
      if check_string == combined_sub_string:
          return " ".join(main_string_list[:i])
  return None

def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
  if len(parse_tree.leaves()) == 1:
      return get_flattened(last_NP),get_flattened(last_VP)
  last_subtree = parse_tree[-1]
  if last_subtree.label() == "NP":
      last_NP = last_subtree
  elif last_subtree.label() == "VP":
      last_VP = last_subtree
  return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)

def get_sentence_completions(key_sentences):
  sentence_completion_dict = {}
  for individual_sentence in key_sentences:
      sentence = individual_sentence.rstrip('?:!.,;')
      tree = benepar_parser.parse(sentence)
      last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
      phrases= []
      if last_verbphrase is not None:
          verbphrase_string = get_termination_portion(sentence,last_verbphrase)
          phrases.append(verbphrase_string)
      if last_nounphrase is not None:
          nounphrase_string = get_termination_portion(sentence,last_nounphrase)
          phrases.append(nounphrase_string)

      longest_phrase =  sorted(phrases, key=len,reverse= True)
      if len(longest_phrase) == 2:
          first_sent_len = len(longest_phrase[0].split())
          second_sentence_len = len(longest_phrase[1].split())
          if (first_sent_len - second_sentence_len) > 4:
              del longest_phrase[1]

      if len(longest_phrase)>0:
          sentence_completion_dict[sentence]=longest_phrase
  return sentence_completion_dict

In [ ]:
def sort_by_similarity(original_sentence,generated_sentences_list):
  model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')
  sentence_embeddings = model_BERT.encode(generated_sentences_list)
  queries = [original_sentence]
  query_embeddings = model_BERT.encode(queries)
  number_top_matches = len(generated_sentences_list)
  dissimilar_sentences = []
  for query, query_embedding in zip(queries, query_embeddings):
      distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]
      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])
      for idx, distance in reversed(results[0:number_top_matches]):
          score = 1-distance
          if score < 0.94: #change
              dissimilar_sentences.append(generated_sentences_list[idx].strip())
  sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
  return sorted_dissimilar_sentences[:3]

def generate_sentences(partial_sentence,full_sentence):
  tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
  model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)
  torch.manual_seed(7879)
  input_ids = torch.tensor([tokenizer.encode(partial_sentence)])
  maximum_length = len(partial_sentence.split())+80
  sample_outputs = model.generate(
      input_ids,
      do_sample=True,
      max_length=maximum_length,
      top_p=0.95, # 0.85
      top_k=50,   #0.30
      repetition_penalty  = 10.0,
      num_return_sequences=10
  )
  generated_sentences=[]
  for i, sample_output in enumerate(sample_outputs):
      decoded_sentences = tokenizer.decode(sample_output, skip_special_tokens=True)
      decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
      generated_sentences.append(decoded_sentences_list[0])
  top_3_sentences = sort_by_similarity(full_sentence,generated_sentences)
  return top_3_sentences

In [ ]:
def trueorfalse(text):
  statements = preprocess(get_candidate_sents(text))
  statement_dict = get_sentence_completions(statements)
  i = 1
  torf = '<p>State whether the following statements are True or False:<br><br>'
  for key_sentence in statement_dict:
    partial_sentences = statement_dict[key_sentence]
    for partial_sent in partial_sentences:
        false_sents = generate_sentences(partial_sent,key_sentence)
    if random.random()<0.5:
      torf += 'Q'+str(i)+') '+key_sentence+'<br>Answer: True<br><br>'
    else:
      torf += 'Q'+str(i)+') '+false_sents[0]+'<br>Answer: False<br><br>'
    i = i+1
  torf += '</p>'
  return torf

# MCQ

In [ ]:
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0:
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

In [ ]:
def get_wordsense(sent,word):
    word= word.lower()
    if len(word.split())>0:
        word = word.replace(" ","_")
    synsets = wn.synsets(word,'n')
    if synsets:
        #wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        #lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        lowest_index = synsets.index(adapted_lesk_output)
        return synsets[lowest_index]
    else:
        return None

In [ ]:
def mcq(text):
  model = summarizer = pipeline("summarization")
  result = model(text, min_length=60, max_length = 500)
  summarized_text = result[0]['summary_text']
  keywords = get_keywords(summarized_text)
  filtered_keys=[]
  for keyword in keywords:
      if keyword.lower() in summarized_text.lower():
          filtered_keys.append(keyword)
  sentences = tokenize_sentences(summarized_text)
  keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
  key_distractor_list = {}
  for keyword in keyword_sentence_mapping:
    if keyword_sentence_mapping[keyword] == []:
      continue
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
  i = 1
  mcq = '<p>Multiple Choice Questions:<br><br>'
  for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    mcq += str(i)+')'+output+'<br>'
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        mcq += optionchoices[idx]+") "+choice+'<br>'
    mcq += 'Answer: '+each+'<br><br>'
    i = i + 1
  mcq += '</p>'
  return mcq

# Match the following

In [ ]:
import os
import zipfile

bert_wsd_pytorch = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6.zip"
extract_directory = "/content/gdrive/My Drive"

extracted_folder = bert_wsd_pytorch.replace(".zip","")

#  If unzipped folder exists don't unzip again.
if not os.path.isdir(extracted_folder):
  with zipfile.ZipFile(bert_wsd_pytorch, 'r') as zip_ref:
      zip_ref.extractall(extract_directory)
else:
  print (extracted_folder," is extracted already")

/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6  is extracted already


In [ ]:
import torch
import math
from transformers import BertModel, BertConfig, BertPreTrainedModel, BertTokenizer

class BertWSD(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.ranking_linear = torch.nn.Linear(config.hidden_size, 1)

        self.init_weights()



DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6"


model = BertWSD.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)
# add new special token
if '[TGT]' not in tokenizer.additional_special_tokens:
    tokenizer.add_special_tokens({'additional_special_tokens': ['[TGT]']})
    assert '[TGT]' in tokenizer.additional_special_tokens
    model.resize_token_embeddings(len(tokenizer))

model.to(DEVICE)
model.eval()

In [ ]:
import csv
import os
from collections import namedtuple

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

import torch
import re
import time
import torch
from tabulate import tabulate
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import BertTokenizer
def _create_features_from_records(records, max_seq_length, tokenizer, cls_token_at_end=False, pad_on_left=False,
                                  cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                  sequence_a_segment_id=0, sequence_b_segment_id=1,
                                  cls_token_segment_id=1, pad_token_segment_id=0,
                                  mask_padding_with_zero=True, disable_progress_bar=False):
    features = []
    for record in tqdm(records, disable=disable_progress_bar):
        tokens_a = tokenizer.tokenize(record.sentence)

        sequences = [(gloss, 1 if i in record.targets else 0) for i, gloss in enumerate(record.glosses)]

        pairs = []
        for seq, label in sequences:
            tokens_b = tokenizer.tokenize(seq)
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
            tokens = tokens_a + [sep_token]
            segment_ids = [sequence_a_segment_id] * len(tokens)

            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)
            padding_length = max_seq_length - len(input_ids)
            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)
            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length
            BertInput = namedtuple("BertInput", ["input_ids", "input_mask", "segment_ids", "label_id"])
            pairs.append(
                BertInput(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_id=label)
            )
        features.append(pairs)
    return features
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def get_sense(sent):
  re_result = re.search(r"\[TGT\](.*)\[TGT\]", sent)
  if re_result is None:
      print("\nIncorrect input format. Please try again.")
  ambiguous_word = re_result.group(1).strip()
  results = dict()
  for i, synset in enumerate(set(wn.synsets(ambiguous_word))):
      results[synset] =  synset.definition()
  if len(results) ==0:
    return None
  sense_keys=[]
  definitions=[]
  for sense_key, definition in results.items():
      sense_keys.append(sense_key)
      definitions.append(definition)
  GlossSelectionRecord = namedtuple("GlossSelectionRecord", ["guid", "sentence", "sense_keys", "glosses", "targets"])
  record = GlossSelectionRecord("test", sent, sense_keys, definitions, [-1])
  MAX_SEQ_LENGTH = 128
  features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                            cls_token=tokenizer.cls_token,
                                            sep_token=tokenizer.sep_token,
                                            cls_token_segment_id=1,
                                            pad_token_segment_id=0,
                                            disable_progress_bar=True)[0]
  with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(DEVICE)
      for i, bert_input in list(enumerate(features)):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
              )[1]
          )
      scores = softmax(logits, dim=0)
      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))
  sense = preds[0][0]
  meaning = preds[0][1]
  return sense

In [ ]:
def matchthefollowing(text):
  keyword_best_sense = {}
  keywords = get_keywords(text)[:8]
  sentences = tokenize_sentences(text)
  keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
  for keyword in  keyword_sentence_mapping:
    try:
      identified_synsets=set(wn.synsets(keyword))
    except:
      continue
    top_3_sentences = keyword_sentence_mapping[keyword][:3]
    best_senses=[]
    for sent in top_3_sentences:
      insensitive_keyword = re.compile(re.escape(keyword), re.IGNORECASE)
      modified_sentence = insensitive_keyword.sub(" [TGT] "+keyword+" [TGT] ", sent,count=1)
      modified_sentence = " ".join(modified_sentence.split())
      best_sense = get_sense(modified_sentence)
      best_senses.append(best_sense)
    if best_senses==[]:
      continue
    best_sense = mode(best_senses)
    if best_sense is not None:
      defn = best_sense.definition()
      keyword_best_sense [keyword] = defn
  all_keywords= list(keyword_best_sense.keys())
  all_definitions = list(keyword_best_sense.values())
  matchthefollow = "<p>Match the following:<br><br><table border=1><tr><th>Column A</th><th>Column B</th></tr>"
  all_d = list(keyword_best_sense.values())
  random.shuffle(all_d)
  ans = "Answer:"
  i,c = 1,'a'
  for key,defn in zip(all_keywords,all_d):
    ans += str(i)+'-'+chr(all_definitions.index(defn)+97)+' '
    matchthefollow += '<tr><td>'+str(i)+') '+key+'</td><td>'+c+') '+defn+'</td></tr>'
    i += 1
    c = chr(ord(c)+1)
  matchthefollow += '</table><br>' + ans + '</p>'
  return matchthefollow

# Fill in the blanks

In [ ]:
def get_fill_in_the_blanks(text):
    sentences = sent_tokenize(text)
    keywords = get_keywords(text)
    sentence_mapping = get_sentences_for_keyword(keywords, sentences)
    out={"title":"Fill in the blanks for these sentences with matching words at the top"}
    blank_sentences = []
    processed = []
    keys=[]
    for key in sentence_mapping:
        if len(sentence_mapping[key])>0:
            sent = sentence_mapping[key][0]
            insensitive_sent = re.compile(re.escape(key), re.IGNORECASE)
            no_of_replacements =  len(re.findall(re.escape(key),sent,re.IGNORECASE))
            line = insensitive_sent.sub(' _________ ', sent)
            if (sentence_mapping[key][0] not in processed) and no_of_replacements<2:
                blank_sentences.append(line)
                processed.append(sentence_mapping[key][0])
                keys.append(key)
    out["sentences"]=blank_sentences
    out["keys"]=keys
    fib = "<p>Fill in the blanks:<br><br>"
    i = 1
    while i<len(out['sentences']):
      fib += "Q"+str(i)+": "+out['sentences'][i-1]+"<br>"
      fib += "Answer: "+out['keys'][i-1]+"<br><br>"
      i += 1
    fib += '</p>'
    return fib

# UI

In [ ]:
def qna(text, questions):
  output = ''
  if 'MCQ' in questions:
    output += mcq(text)
  if 'True/False' in questions:
    output += trueorfalse(text)
  if 'Fill in the Blanks' in questions:
    output += get_fill_in_the_blanks(text)
  if 'Match the following' in questions:
    output += matchthefollowing(text)
  return output
iface = gr.Interface(
  qna,
  [gr.inputs.Textbox(lines=10, placeholder="Enter text here"),
  gr.inputs.CheckboxGroup(['MCQ', 'True/False', 'Fill in the Blanks', 'Match the following'], label="Select the type of questions"),],
  outputs=gr.outputs.HTML(label="Question and Answers"),
  allow_screenshot=False,
  allow_flagging="never",
  title="Question Paper Generator",)
iface.launch(debug=False)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://20539.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f35c7826c90>,
 'http://127.0.0.1:7860/',
 'https://20539.gradio.app')

# Run

In [ ]:
text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""

In [ ]:
#Fill in the blanks
fill_in_the_blanks = get_fill_in_the_blanks(text)
print(fill_in_the_blanks)

<p>Fill in the blanks:<br><br>Q1:  _________  boundaries also occur in the continental crust.<br>Answer: divergent plate<br><br>Q2: The leading edge of the  _________  melts as it is pulled into the mantle, forming magma that erupts as volcanoes.<br>Answer: plate<br><br>Q3:  _________  form at these boundaries, but less often than in ocean crust.<br>Answer: volcanoes<br><br>Q4: Many volcanoes form along convergent  _________  where one tectonic plate is pulled down beneath another at a subduction zone.<br>Answer: plate boundaries<br><br>Q5: This is a divergent plate boundary that runs north-south through the middle of the  _________ .<br>Answer: Atlantic Ocean<br><br>Q6: At the surface, the  _________  is called lava.<br>Answer: Molten rock<br><br>Q7: It cools and hardens, forming  _________ .<br>Answer: rock<br><br>Q8: The edges of the Pacific plate are long  _________  zones lined with volcanoes.<br>Answer: subduction<br><br>Q9: For example, many undersea volcanoes are found along th

In [ ]:
#Match the following
mf = matchthefollowing(text)
print(mf)

In [ ]:
#MCQ
print(mcq(text))

<p>Multiple Choice Questions:<br><br>1)For example, many undersea volcanoes are found along the Mid-Atlantic  _______ .<br>a) Girder<br>b) Box Beam<br>c) Ridge<br>d) Cantilever<br>Answer: Ridge<br><br>2)That’s because continental  _______  is thicker than oceanic  _______ .<br>a) Abruptness<br>b) Boorishness<br>c) Contempt<br>d) Crust<br>Answer: crust<br><br>3)There is a  _______  of volcanic activity at divergent plate boundaries in the oceans.<br>a) Catch<br>b) Charm<br>c) Commemorative<br>d) Lot<br>Answer: lot<br><br>4)There is a lot of volcanic  _______  at divergent plate boundaries in the oceans.<br>a) Chelation<br>b) Decrease<br>c) Activity<br>d) Dealignment<br>Answer: activity<br><br>5)Volcanoes form at these  _______ , but less often than in ocean crust.<br>a) Boundaries<br>b) Area<br>c) Depth<br>d) Coverage<br>Answer: boundaries<br><br>6) _______  form at these boundaries, but less often than in ocean crust.<br>a) Ben<br>b) Seamount<br>c) Volcanoes<br>d) Alp<br>Answer: volcan

In [ ]:
#True or False
print(trueorfalse(text))

In [ ]:
text = "The Greek historian knew what he was talking about. The Nile river fed Egyptian civilization for hundreds of years. The longest river the Nile is 4,160 miles long—the world’s longest river. It begins near the equator in Africa and flows north to the Mediterranean Sea. In the south the Nile churns with cataracts. A cataract is a waterfall. Near the sea the Nile branches into a delta. A delta is an area near a river’s mouth where the water deposits fine soil called silt. In the delta, the Nile divides into many streams. The river is called the upper Nile in the south and the lower Nile in the north. For centuries, heavy rains in Ethiopia caused the Nile to flood every summer. The floods deposited rich soil along the Nile’s shores. This soil was fertile, which means it was good for growing crops. Unlike the Tigris and Euphrates, the Nile River flooded at the same time every year, so farmers could predict when to plant their crops. Red Land, Black Land The ancient Egyptians lived in narrow bands of land on each side of the Nile. They called this region the black land because of the fertile soil that the floods deposited. The red land was the barren desert beyond the fertile region. Weather in Egypt was almost always the same. Eight months of the year were sunny and hot. The four months of winter were sunny but cooler. Most of the region received only an inch of rain a year. The parts of Egypt not near the Nile were a desert. Isolation The harsh desert acted as a barrier to keep out enemies. The Mediterranean coast was swampy and lacked good harbors. For these reasons, early Egyptians stayed close to home. Each year, Egyptian farmers watched for white birds called ibises, which flew up from the south. When the birds arrived, the annual flood waters would soon follow. After the waters drained away, farmers could plant seeds in the fertile soil. Agricultural Techniques By about 2400 B.C., farmers used technology to expand their farmland. Working together, they dug irrigation canals that carried river water to dry areas. Then they used a tool called a shaduf to spread the water across the fields. These innovative, or new, techniques gave them more farmland. Egyptian Crops Ancient Egyptians grew a large variety of foods. They were the first to grind wheat into flour and to mix the flour with yeast and water to make dough rise into bread. They grew vegetables such as lettuce, radishes, asparagus, and cucumbers. Fruits included dates, figs, grapes, and watermelons. Egyptians also grew the materials for their clothes. They were the first to weave fibers from flax plants into a fabric called linen. Lightweight linen cloth was perfect for hot Egyptian days. Men wore linen wraps around their waists. Women wore loose, sleeveless dresses. Egyptians also wove marsh grasses into sandals. Egyptian houses Egyptians built houses using bricks made of mud from the Nile mixed with chopped straw. They placed narrow windows high in the walls to reduce bright sunlight. Egyptians often painted walls white to reflect the blazing heat. They wove sticks and palm trees to make roofs. Inside, woven reed mats covered the dirt floor. Most Egyptians slept on mats covered with linen sheets. Wealthy citizens enjoyed bed frames and cushions. Egyptian nobles had fancier homes with tree-lined courtyards for shade. Some had a pool filled with lotus blossoms and fish. Poorer Egyptians simply went to the roof to cool off after sunset. They often cooked, ate, and even slept outside. Egypt’s economy depended on farming. However, the natural resources of the area allowed other economic activities to develop too. The Egyptians wanted valuable metals that were not found in the black land. For example, they wanted copper to make tools and weapons. Egyptians looked for copper as early as 6000 B.C. Later they learned that iron was stronger, and they sought it as well. Ancient Egyptians also desired gold for its bright beauty. The Egyptian word for gold was nub. Nubia was the Egyptian name for the area of the upper Nile that had the richest gold mines in Africa. Mining minerals was difficult. Veins (long streaks) of copper, iron, and bronze were hidden inside desert mountains in the hot Sinai Peninsula, east of Egypt. Even during the cool season, chipping minerals out of the rock was miserable work. Egyptians mined precious stones too. They were probably the first people in the world to mine turquoise. The Egyptians also mined lapis lazuli. These beautiful blue stones were used in jewelry.The Nile had fish and other wildlife that Egyptians wanted. To go on the river, Egyptians made lightweight rafts by binding together reeds. They used everything from nets to harpoons to catch fish. One ancient painting even shows a man ready to hit a catfish with a wooden hammer. More adventurous hunters speared hippopotamuses and crocodiles along the Nile. Egyptians also captured quail with nets. They used boomerangs to knock down flying ducks and geese. (A boomerang is a curved stick that returns to the person who threw it.) Eventually, Egyptians equipped their reed boats with sails and oars. The Nile then became a highway. The river’s current was slow, so boaters used paddles to go faster when they traveled north with the current. Going south, they raised a sail and let the winds that blew in that direction push them. The Nile provided so well for Egyptians that sometimes they had surpluses, or more goods than they needed. They began to trade with each other. Ancient Egypt had no money, so people exchanged goods that they grew or made. This method of trade is called bartering. Egypt prospered along the Nile. This prosperity made life easier and provided greater opportunities for many Egyptians. When farmers produce food surpluses, the society’s economy begins to expand. Cities emerge as centers of culture and power, and people learn to do jobs that do not involve agriculture. For example, some ancient Egyptians learned to be scribes, people whose job was to write and keep records. As Egyptian civilization grew more complex, people took on jobs other than that of a farmer or scribe. Some skilled artisans erected stone or brick houses and temples. Other artisans made pottery, incense, mats, furniture, linen clothing, sandals, or jewelry. A few Egyptians traveled to the upper Nile to trade with other Africans. These traders took Egyptian products such as scrolls, linen, gold, and jewelry. They brought back exotic woods, animal skins, and live beasts. As Egypt grew, so did its need to organize. Egyptians created a government that divided the empire into 42 provinces. Many officials worked to keep the provinces running smoothly. Egypt also created an army to defend itself. One of the highest jobs in Egypt was to be a priest. Priests followed formal rituals and took care of the temples. Before entering a temple, a priest bathed and put on special linen garments and white sandals. Priests cleaned the sacred statues in temples, changed their clothes, and even fed them meals. Together, the priests and the ruler held ceremonies to please the gods. Egyptians believed that if the gods were angry, the Nile would not flood. As a result, crops would not grow, and people would die. So the ruler and the priests tried hard to keep the gods happy. By doing so, they hoped to maintain the social and political order. Slaves were at the bottom of society. In Egypt, people became slaves if they owed a debt, committed a crime, or were captured in war. Egyptian slaves were usually freed after a period of time. One exception was the slaves who had to work in the mines. Many died from the exhausting labor. Egypt was one of the best places in the ancient world to be a woman. Unlike other ancient African cultures, in Egyptian society men and women had fairly equal rights. For example, they could both own and manage their own property. The main job of most women was to care for their children and home, but some did other jobs too. Some women wove cloth. Others worked with their husbands in fields or workshops. Some women, such as Queen Tiy, even rose to important positions in the government. Children in Egypt played with toys such as dolls, animal figures, board games, and marbles. Their parents made the toys from wood or clay. Boys and girls also played rough physical games with balls made of leather or reeds. Boys and some girls from wealthy families went to schools run by scribes or priests. Most other children learned their parents’ jobs. Almost all Egyptians married when they were in their early teens. As in many ancient societies, much of the knowledge of Egypt came about as priests studied the world to find ways to please the gods. Other advances came about because of practical discoveries. Egyptian priests studied the sky as part of their religion. About 5,000 years ago, they noticed that a star now called Sirius appeared shortly before the Nile began to flood. The star returned to the same position in 365 days. Based on that, Egyptians developed the world’s first practical calendar. The Egyptians developed some of the first geometry. Each year the Nile’s floods washed away land boundaries. To restore property lines, surveyors measured the land by using ropes that were knotted at regular intervals. Geometric shapes such as squares and triangles were sacred to Egyptians. Architects used them in the design of royal temples and monuments. Egyptian doctors often prepared dead bodies for burial, so they knew the parts of the body. That knowledge helped them perform some of the world’s first surgery. Some doctors specialized in using medicines made of herbs. Egyptian medicine was far from perfect. Doctors believed that the heart controlled thought and the brain circulated blood, which is the opposite of what is known now. Some Egyptian treatments would raise eyebrows today. One “cure” for an upset stomach was to eat a hog’s tooth crushed inside sugar cakes! Beginning about 3000 B.C., Egyptians developed a writing system using hieroglyphs. Hieroglyphs Hieroglyphs are pictures that stand for different words or sounds. Early Egyptians created a hieroglyphic system with about 700 characters. Over time the system grew to include more than 6,000 symbols. The Egyptians also developed a paperlike material called papyrus papyrus from a reed of the same name. Egyptians cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls. Papyrus scrolls were light and easy to carry. With them, Egyptians created some of the first books. Legend says a king named Narmer united Upper and Lower Egypt. Some historians think Narmer actually represents several kings who gradually joined the two lands. After Egypt was united, its ruler wore the Double Crown. It combined the red Crown of Lower Egypt with the white Crown of Upper Egypt. The first dynasty of the Egyptian empire began about 2925 B.C. A dynasty is a line of rulers from the same family. When a king died, one of his children usually took his place as ruler. The order in which members of a royal family inherit a throne is called the succession. More than 30 dynasties ruled ancient Egypt. Historians divide ancient Egyptian dynasties into the Old Kingdom, the Middle Kingdom, and the New Kingdom. The Old Kingdom started about 2575 B.C., when the Egyptian empire was gaining strength. The king of Egypt became known as the pharaoh pharaoh. The word pharaoh meant “great house,” and it was originally used to describe the king’s palace. Later it became the title of the king himself. The pharaoh ruled from the capital city of Memphis. The ancient Egyptians thought the pharaoh was a child of the gods and a god himself. Egyptians believed that if the pharaoh and his subjects honored the gods, their lives would be happy. If Egypt suffered hard times for a long period, the people blamed the pharaoh for angering the gods. In such a case, a rival might drive him from power and start a new dynasty. Because the pharaoh was thought to be a god, government and religion were not separate in ancient Egypt. Priests had much power in the government. Many high officials were priests. The first rulers of Egypt were often buried in an underground tomb topped by mud brick. Soon, kings wanted more permanent monuments. They replaced the mud brick with a small pyramid of brick or stone. A pyramid is a structure shaped like a triangle, with four sides that meet at a point. About 2630 B.C., King Djoser built a much larger pyramid over his tomb. It is called a step pyramid because its sides rise in a series of giant steps. It is the oldest-known large stone structure in the world. About 80 years later, a pharaoh named Khufu decided he wanted a monument that would show the world how great he was. He ordered the construction of the largest pyramid ever built. Along its base, each side was about 760 feet long. The core was built from 2.3 million blocks of stone. Building the Great Pyramid was hard work. Miners cut the huge blocks of stone using copper saws and chisels. These tools were much softer than the iron tools developed later. Other teams of workers pulled the stone slabs up long, sloping ramps to their place on the pyramid. Near the top of the pyramid, the ramps ended. Workers dragged each heavy block hundreds of feet and then set it in place. Farmers did the heavy labor of hauling stone during the season when the Nile flooded their fields. Skilled stonecutters and overseers worked year-round. The Great Pyramid took nearly 20 years to build. An estimated 20,000 Egyptians worked on it. A city called Giza was built for the pyramid workers and the people who fed, clothed, and housed them. Eventually, Egyptians stopped building pyramids. One reason is that the pyramids drew attention to the tombs inside them. Grave robbers broke into the tombs to steal the treasure buried with the pharaohs. Sometimes they also stole the mummies. Egyptians believed that if a tomb was robbed, the person buried there could not have a happy afterlife. During the New Kingdom, pharaohs began building more secret tombs in an area called the Valley of the Kings. The burial chambers were hidden in mountains near the Nile. This way, the pharaohs hoped to protect their bodies and treasures from robbers. Both the pyramids and later tombs had several passageways leading to different rooms. This was to confuse grave robbers about which passage to take. Sometimes relatives, such as the queen, were buried in the extra rooms. Tombs were supposed to be the palaces of pharaohs in the afterlife. Mourners filled the tomb with objects ranging from food to furniture that the mummified pharaoh would need. Some tombs contained small statues that were supposed to be servants for the dead person. Egyptian artists decorated royal tombs with wall paintings and sculptures carved into the walls. Art was meant to glorify both the gods and the dead person. A sculpture of a dead pharaoh had “perfect” features, no matter how he really looked. Artists also followed strict rules about how to portray humans. Paintings showed a person’s head, arms, and legs from the side. They showed the front of the body from the neck down to the waist. Wall paintings showed pharaohs enjoying themselves so they could have a happy afterlife. One favorite scene was of the pharaoh fishing in a papyrus marsh. Warlike kings were often portrayed in battle. Scenes might also show people providing for the needs of the dead person. Such activities included growing and preparing food, caring for animals, and building boats. As hard as the pharaohs tried to hide themselves, robbers stole the treasures from almost every tomb. Only a secret tomb built for a New Kingdom pharaoh was ever found with much of its treasure untouched. The dazzling riches found in this tomb show how much wealth the pharaohs spent preparing for the afterlife. By about 2130 B.C., Egyptian kings began to lose their power to local rulers of the provinces. For about 500 more years, the kings held Egypt together, but with a much weaker central government. This period of Egyptian history is called the Middle Kingdom. Rulers during the Middle Kingdom also faced challenges from outside Egypt. A nomadic people called the Hyksos invaded Egypt from the northeast. Their army conquered by using better weapons and horse-drawn chariots, which were new to Egyptians. After about 100 years, the Egyptians drove out the Hyksos and began the New Kingdom."